**Imports**


---



In [ ]:
!apt install fluidsynth
!cp /usr/share/sounds/sf2/FluidR3_GM.sf2 ./font.sf2

from music21 import *
from IPython.display import Audio
import numpy as np

Reading package lists... Done
Building dependency tree       
Reading state information... Done
fluidsynth is already the newest version (1.1.9-1).
0 upgraded, 0 newly installed, 0 to remove and 14 not upgraded.


In [ ]:
validation_data = np.load('validation_prediction.npy')
validation_data

In [91]:
input_data = np.load('shuffled_validation_data.npy')
input_data = np.argmax(input_data, axis=-1)
input_data

array([[129, 129, 129, ..., 129, 129, 129],
       [ 45,  60,  69, ...,  59,  60,  69],
       [ 73, 128,  69, ..., 128, 128, 128],
       ...,
       [ 42,  62,  50, ...,  59, 129, 129],
       [129, 129, 129, ..., 129, 129, 129],
       [ 57, 128,  64, ..., 129, 129, 129]])

**Helper functions**


---



In [123]:
REST_VALUE = 129
HOLD_VALUE = 128

# Commits a note to the stream
def commit(s, lastHeardNote, lastHeardNoteLength):
  # if your c0 is 0 in the array instead of the standard, add the offset here
  # this is added to the value of the index when creating the midi note
  if lastHeardNoteLength > 2.0:
    print("(LONG NOTE WTF)")
  PITCH_OFFSET = 0 

  if lastHeardNote == HOLD_VALUE:
    raise Exception("Cannot commit a HOLD to the stream (only rests or pitched notes)")

  if lastHeardNote == REST_VALUE:
    newNote = note.Rest()
  else:
    newNote = note.Note(lastHeardNote + PITCH_OFFSET)
  newNote.quarterLength = lastHeardNoteLength
  
  if lastHeardNote == REST_VALUE: # if this is a rest
    newNote.isRest = True
  else:
    newNote.isRest = False

  print(f'Added {newNote}')
  s.append(newNote)
  return s

def decode_midi_include_inputs(data, max_2bar_segments=20):
  UNINITIALIZED_VALUE = -1 # for the last heard note

  print("Begin decoding input...")
  print(data)
  print(data.shape)

  # double the bar count since we're selecting from input and output
  max_2bar_segments = max_2bar_segments * 2
    
  # select max number of bars
  data = data[:max_2bar_segments]
  input = input_data[:max_2bar_segments]

  combined_data = zip(input, data)

  s = stream.Stream()
  lastHeardNote = UNINITIALIZED_VALUE
  lastHeardLength = 0 # 4 for a full beat
  two_bar_count = 0
  for twoBars_in, twoBars_out in combined_data:
    if two_bar_count > max_2bar_segments:
      break

    for sixteenthNote in twoBars_in:
      # is this the first note in the song? then don't worry about holds
      if lastHeardNote == UNINITIALIZED_VALUE:
        lastHeardNote = sixteenthNote
        lastHeardLength = 0.25 # one sixteenth note
      else: # any note but the first
        if sixteenthNote == HOLD_VALUE: # hold
          lastHeardLength = lastHeardLength + 0.25
        elif sixteenthNote == REST_VALUE: # this is a rest
          # commit the last note
          s = commit(s, lastHeardNote, lastHeardLength)
          lastHeardNote = REST_VALUE
          lastHeardLength = 0.25
        else: # this is any other note
          # commit whatever the last note was
          # TODO: this looks equivalent to the elif above, consider condensing
          s = commit(s, lastHeardNote, lastHeardLength)
          lastHeardNote = sixteenthNote
          lastHeardLength = 0.25

    for sixteenthNote in twoBars_out:
      # is this the first note in the song? then don't worry about holds
      if lastHeardNote == UNINITIALIZED_VALUE:
        lastHeardNote = sixteenthNote
        lastHeardLength = 0.25 # one sixteenth note
      else: # any note but the first
        if sixteenthNote == HOLD_VALUE: # hold
          lastHeardLength = lastHeardLength + 0.25
        elif sixteenthNote == REST_VALUE: # this is a rest
          # commit the last note
          s = commit(s, lastHeardNote, lastHeardLength)
          lastHeardNote = REST_VALUE
          lastHeardLength = 0.25
        else: # this is any other note
          # commit whatever the last note was
          # TODO: this looks equivalent to the elif above, consider condensing
          s = commit(s, lastHeardNote, lastHeardLength)
          lastHeardNote = sixteenthNote
          lastHeardLength = 0.25
          
    two_bar_count = two_bar_count + 1
  s.show('text')
  return s

def decode_midi(data, max_2bar_segments=20, shuffle=False):
  UNINITIALIZED_VALUE = -1 # for the last heard note

  print("Begin decoding input...")
  print(data)
  print(data.shape)

  if shuffle == True:
    print("Shuffling the data...")
    np.random.shuffle(data)
    
  # select max number of bars
  data = data[:max_2bar_segments]

  print(f'data shape: {data.shape}')

  s = stream.Stream()
  lastHeardNote = UNINITIALIZED_VALUE
  lastHeardLength = 0 # 4 for a full beat
  two_bar_count = 0
  for twoBars in data:
    if two_bar_count > max_2bar_segments:
      break

    print(twoBars)
    for sixteenthNote in twoBars:
      # is this the first note in the song? then don't worry about holds
      if lastHeardNote == UNINITIALIZED_VALUE:
        lastHeardNote = sixteenthNote
        lastHeardLength = 0.25 # one sixteenth note
      else: # any note but the first
        if sixteenthNote == HOLD_VALUE: # hold
          lastHeardLength = lastHeardLength + 0.25
        elif sixteenthNote == REST_VALUE: # this is a rest
          # commit the last note
          s = commit(s, lastHeardNote, lastHeardLength)
          lastHeardNote = REST_VALUE
          lastHeardLength = 0.25
        else: # this is any other note
          # commit whatever the last note was
          # TODO: this looks equivalent to the elif above, consider condensing
          s = commit(s, lastHeardNote, lastHeardLength)
          lastHeardNote = sixteenthNote
          lastHeardLength = 0.25
    two_bar_count = two_bar_count + 1
  s.show('text')
  return s

**Create MIDI**

---



In [125]:
# normalize predictions to indexed values
encoded_midi = np.argmax(validation_data, axis=-1)

# generate random MIDI passage
#outputStream = decode_midi(encoded_midi, max_2bar_segments=20, shuffle=True)

# generate two bars of input and two bars of predicted output
outputStream = decode_midi_include_inputs(encoded_midi, 5)

Begin decoding input...
[[129 129 129 ... 129 129 129]
 [ 40  64  67 ...  60  60  65]
 [ 73 128  71 ... 128 128 128]
 ...
 [ 50  62  45 ...  66  59  66]
 [129 129 129 ... 129 129 129]
 [ 57 128  64 ... 129 129 129]]
(9353, 32)
Added <music21.note.Rest rest>
Added <music21.note.Rest rest>
Added <music21.note.Rest rest>
Added <music21.note.Rest rest>
Added <music21.note.Rest rest>
Added <music21.note.Rest rest>
Added <music21.note.Rest rest>
Added <music21.note.Rest rest>
Added <music21.note.Note E>
Added <music21.note.Rest rest>
Added <music21.note.Rest rest>
Added <music21.note.Rest rest>
Added <music21.note.Rest rest>
Added <music21.note.Note A>
Added <music21.note.Rest rest>
Added <music21.note.Rest rest>
Added <music21.note.Rest rest>
Added <music21.note.Rest rest>
Added <music21.note.Rest rest>
Added <music21.note.Rest rest>
Added <music21.note.Rest rest>
Added <music21.note.Rest rest>
Added <music21.note.Rest rest>
Added <music21.note.Rest rest>
Added <music21.note.Rest rest>
Adde

**Play generated MIDI**


---



In [126]:
midiFile = midi.translate.streamToMidiFile(outputStream)
midiFile.open('input.mid', 'wb')
midiFile.write()
midiFile.close()

!fluidsynth -ni SpanishGuitar.sf2 input.mid -F output.wav -r 44100
Audio('output.wav')


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# Sandbox
